# PyTorch_Learningn (MIVRC)

![](https://ws1.sinaimg.cn/large/abaebc48ly1fqgrsuu355j209x02wjr7.jpg)

## 课程5：前馈神经网络

### 代码示例：

In [1]:
#导入所需的库/包
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#定义超参数
# input_size: 输入层维度
# hidden_size: 隐藏层维度
# num_class: 类别
input_size = 784
hidden_size = 500
num_class = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [3]:
#下载MNIST 数据集
train_dataset = datasets.MNIST(root='./MNIST', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./MNIST', train=False, transform=transforms.ToTensor(), download=True)

In [4]:
#加载数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
#构建前馈神经网络模型
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_class):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_class)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
#加载网络模型，并调用GPU加速
net = Net(input_size, hidden_size, num_class)
net.cuda()

Net (
  (relu): ReLU ()
  (fc1): Linear (784 -> 500)
  (fc2): Linear (500 -> 10)
)

In [7]:
#定义损失函数
criterion = nn.CrossEntropyLoss()
#定义优化器
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [8]:
#开始训练网络
#.cuda() 代表调用GPU加速
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28)).cuda()
        labels = Variable(labels).cuda()
        
        #调用模型进行前向传播
        outputs = net(images)
        
        #调用损失函数，计算输出值和真实值之间的差
        loss = criterion(outputs, labels)
        
        #启用优化器并将梯度缓存清空，再进行反向传播
        optimizer.zero_grad()
        loss.backward()
    
        #调用优化函数开始迭代优化
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/10], Step [100/600], Loss: 0.2927
Epoch [1/10], Step [200/600], Loss: 0.2754
Epoch [1/10], Step [300/600], Loss: 0.1957
Epoch [1/10], Step [400/600], Loss: 0.1892
Epoch [1/10], Step [500/600], Loss: 0.1402
Epoch [1/10], Step [600/600], Loss: 0.1370
Epoch [2/10], Step [100/600], Loss: 0.0878
Epoch [2/10], Step [200/600], Loss: 0.0619
Epoch [2/10], Step [300/600], Loss: 0.1621
Epoch [2/10], Step [400/600], Loss: 0.1861
Epoch [2/10], Step [500/600], Loss: 0.0505
Epoch [2/10], Step [600/600], Loss: 0.0813
Epoch [3/10], Step [100/600], Loss: 0.0745
Epoch [3/10], Step [200/600], Loss: 0.0850
Epoch [3/10], Step [300/600], Loss: 0.1668
Epoch [3/10], Step [400/600], Loss: 0.0945
Epoch [3/10], Step [500/600], Loss: 0.0370
Epoch [3/10], Step [600/600], Loss: 0.0385
Epoch [4/10], Step [100/600], Loss: 0.0553
Epoch [4/10], Step [200/600], Loss: 0.0816
Epoch [4/10], Step [300/600], Loss: 0.0714
Epoch [4/10], Step [400/600], Loss: 0.0416
Epoch [4/10], Step [500/600], Loss: 0.0474
Epoch [4/10

In [9]:
#测试网络性能
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28)).cuda()
    outputs = net(images)
    #torch.max(x, n) 沿着n维进行某种操作,即取10类概率最大的结果作为最后的输出
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [10]:
#保存网络模型及参数
torch.save(net.state_dict(), 'Net.pkl')